In [ ]:
import os
import sys
sys.path.append('../src')

import numpy as np
from texttable import Texttable
import latextable

from metrics import print_and_analysis_performance_mean_std, print_performance_mean_std, print_overall_performance_mean_std, print_ablation_performance_mean_std

In [ ]:
dropout = 0.5
alpha = 1
seeds = [10, 20, 30, 40, 50]
upset_choices = ['upset simple', 'upset ratio', 'upset naive']
selected_metrics = ['kendall tau'] + upset_choices
NUM_UPSET_CHOICES = len(upset_choices)
METRICS_NUM = len(selected_metrics)
baselines = ['SpringRank','syncRank','serialRank','btl', 'davidScore',
        'eigenvectorCentrality', 'PageRank', 'rankCentrality', 'SVD_RS', 'SVD_NRS']
NUM_BASELINES = len(baselines)
GNN_selection_choices = ['lr', 'train with', 'pretrain with', 'upset margin coeff', \
                'trainable alpha', 'baseline', \
                         'upset ratio coeff', 'Fiedler layer num', 'pretrain epochs']
GNN_selection_choices_curr = ['train with', 'pretrain with', 'upset margin coeff',  \
                 'baseline', 'upset ratio coeff']
GNN_CHOICES_NUM = len(GNN_selection_choices)
GNN_CHOICES_NUM_CURR = len(GNN_selection_choices_curr)
mvr = ['mvr']
all_GNNs = ['DIGRAC', 'ib']
desirable_list = [1, 2, 5]
train_with_list = ['dist', 'innerproduct', 'proximal_dist', 'proximal_innerproduct','proximal_baseline']

GNN_variant_names = train_with_list
GNN_NUM = 10

def generate_method_str_and_compare_names_all(all_methods=baselines):
    method_str = ''
    for method_name in all_methods:
        method_str += method_name
    compare_names_all = []
    for method_name in all_methods:
        if method_name not in ['DIGRAC', 'ib']:
            compare_names_all.append(method_name)
        else:
            for GNN_type in GNN_variant_names:
                compare_names_all.append(method_name+'_'+GNN_type)
    return method_str, compare_names_all

methods_of_interest = ['Non_proximal', 'Proximal']
        
GNN_names = []
for method_name in ['DIGRAC', 'ib']:
    for GNN_type in GNN_variant_names:
        GNN_names.append(method_name+'_'+GNN_type)

non_proximal_ind = [0, 1, 5, 6]
proximal_ind = [2, 3, 4, 7, 8, 9]
NON_PROXIMAL_GNN_NUM = len(non_proximal_ind)
PROXIMAL_GNN_NUM = len(proximal_ind)
non_proximal_bool = np.zeros(GNN_NUM, dtype=bool)
proximal_bool = np.zeros(GNN_NUM, dtype=bool)
for i in non_proximal_ind:
    non_proximal_bool[i] = True
for i in proximal_ind:
    proximal_bool[i] = True

GNN_names_non_proximal = ['DIGRAC_dist', 'DIGRAC_innerproduct',\
                          'ib_dist', 'ib_innerproduct']
GNN_names_proximal = ['DIGRAC_proximal_dist', 'DIGRAC_proximal_innerproduct', 'DIGRAC_proximal_baseline', \
                      'ib_proximal_dist', 'ib_proximal_innerproduct', 'ib_proximal_baseline']


compare_names_all = baselines + mvr + methods_of_interest
METHODS_NUM = len(compare_names_all)

In [ ]:
keys = ['DIGRAC_dist', 'DIGRAC_innerproduct',\
                          'ib_dist', 'ib_innerproduct', \
        'DIGRAC_proximal_dist', 'DIGRAC_proximal_innerproduct', 'DIGRAC_proximal_baseline', \
                      'ib_proximal_dist', 'ib_proximal_innerproduct', 'ib_proximal_baseline', \
       ]
values = ['DIGRAC dist', 'DIGRAC innerproduct',\
                          'ib dist', 'ib innerproduct', \
        'DIGRAC proximal dist', 'DIGRAC proximal innerproduct', 'DIGRAC proximal baseline', \
                      'ib proximal dist', 'ib proximal innerproduct', 'ib proximal baseline']

keys += train_with_list + ['dist', 'innerproduct', 'serial_similarity'] + \
['avg_football', 'avg_finer_football', 'avg_basketball', 'avg_finer_basketball'] + \
['serialRank', 'SpringRank','syncRank','btl', 'eigenvectorCentrality', 'PageRank', 'SVD_NRS']
values += ['dist', 'innerproduct', 'proximal dist', 'proximal innerproduct','proximal baseline'] + \
['dist', 'innerproduct', 'SerialRank similarity'] + \
['{\it Football (avg)}', '{\it Football finer (avg)}', '{\it Basketball (avg)}', '{\it Basketball finer (avg)}'] + \
['SerialRank', 'SpringRank','SyncRank','BTL','Eign.Cent.', 'PageRank', 'SVD\_NRS']
name_mapping_dict = dict(zip(keys, values))
print(name_mapping_dict)

In [ ]:
p_list_dict = {}
eta_list_dict = {}
ERO_style_list_dict = {}
K_list_dict = {}
season_list_dict = {}
season_list_dict['basketball'] = np.arange(1985, 2015)
season_list_dict['finer_basketball'] = np.arange(1985, 2015)
season_list_dict['football'] = np.arange(2009, 2015)
season_list_dict['finer_football'] = np.arange(2009, 2015)
for dataset in ['finance','animal', 'faculty_business', 'faculty_cs', 'faculty_history', 'HeadToHead', 'ERO']:
    season_list_dict[dataset] = [2009]
for dataset in ['basketball', 'finer_basketball', 'football', 'finer_football', 'finance','animal', 'faculty_business', 'faculty_cs', 'faculty_history', 'HeadToHead']:
    p_list_dict[dataset] = [0.05]
    eta_list_dict[dataset] = [0]
    K_list_dict[dataset] = [5]
    ERO_style_list_dict[dataset] = ['uniform']

p_list_dict['ERO'] = [0.05, 1]
eta_list_dict['ERO'] = [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8]
K_list_dict['ERO'] = [5]
ERO_style_list_dict['ERO'] = ['uniform', 'gamma']
pretrain_with_list_dict = {}
for train_with in train_with_list:
    if train_with[:8] == 'proximal':
        pretrain_with_list_dict[train_with] = ['dist', 'innerproduct', 'serial_similarity']
    else:
        pretrain_with_list_dict[train_with] = ['dist']

In [ ]:
def generate_save_name(dataset='HeadToHead', all_methods=all_GNNs, K=5, train_with='dist', upset_ratio_coeff=1.0, upset_margin_coeff=0, upset_margin=0.01,
                           trainable_alpha=False, lr=0.01, hidden=32, num_trials=10, train_ratio=1, test_ratio=1,  AllTrain=True, rank_baseline='SpringRank', sigma=1.0, 
                           Fiedler_layer_num=5, pretrain_epochs=50, pretrain_with='dist'):
    default_name_base = ''
    if 'DIGRAC' in all_methods or 'ib' in all_methods:
        default_name_base += 'K' + str(K) + 'dropout' + str(int(100*dropout))
        default_name_base += 'ratio_coe' + str(int(100*upset_ratio_coeff)) + 'margin_coe' + str(int(100*upset_margin_coeff)) 
        if upset_margin_coeff > 0:
            default_name_base += 'margin' + str(int(100*upset_margin)) 
        default_name_base += 'with' + str(train_with)  + 'Fiedler' + str(Fiedler_layer_num) + 'sigma' + str(int(100*sigma))
        default_name_base += 'alpha' + str(int(100*alpha))
        if train_with[:8] == 'proximal':
            default_name_base += 'train_alpha' + str(trainable_alpha)
        default_name_base += 'hid' + str(hidden) + 'lr' + str(int(1000*lr))
        default_name_base += 'use' + str(rank_baseline)
        if pretrain_epochs > 0 and train_with[:8] == 'proximal':
            default_name_base +=  'pre' + str(pretrain_with) + str(int(pretrain_epochs))
    save_name_base = default_name_base

    default_name_base +=  'trials' + str(num_trials) + 'train_r' + str(int(100*train_ratio)) + 'test_r' + str(int(100*test_ratio)) + 'All' + str(AllTrain)
    if dataset[:3] == 'ERO':
        default_name_base += 'seeds' + '_'.join([str(value) for value in np.array(seeds).flatten()])
    return default_name_base

In [ ]:
def dataset_and_K(dataset, season=2009, K=5, p=0.05, ERO_style='uniform', eta=0.1, N=350):
    F_style = 'path'
    sp_style = 'random'
    ambient = 0
    if dataset[-1]!='/':
        dataset += '/'

    if dataset[:3] == 'ERO':
        hidden = 8
        K = 5
        F = 3
        default_name_base = 'p' + str(int(100*p)) + 'K' + str(K) + 'N' + str(N)
        default_name_base += 'eta' + str(int(100*eta)) + 'style' + str(ERO_style)
        dataset = 'ERO/' + default_name_base
        dataset_print = 'ERO(p={}, style={},$\eta$={})'.format(p, ERO_style, eta)
    elif dataset[:10].lower() == 'basketball':
        hidden = 8
        hidden_compare = 8
        F = 70
        K = 20
        dataset = 'Basketball_temporal/' + str(season)
        dataset_print = 'Basketball({})'.format(season)
    elif dataset[:16].lower() == 'finer_basketball':
        hidden = 8
        hidden_compare = 8
        F = 2
        K = 20
        dataset = 'Basketball_temporal/finer' + str(season)
        dataset_print = 'Basketball finer({})'.format(season)
    elif dataset[:6].lower() == 'animal':
        hidden = 4
        hidden_compare = 4
        F = 3
        K = 3
        dataset = 'Dryad_animal_society/'
        dataset_print = 'Animal'
    elif dataset[:7].lower() == 'finance':
        hidden = 32
        hidden_compare = 32
        F = 5 # threshold: > 0.7, others have threshold > 0.9
        K = 20
        dataset_print = 'Finance'
    elif dataset[:10].lower() == 'headtohead':
        hidden = 16
        hidden_compare = 16
        F = 39
        K = 48
        dataset = 'Halo2BetaData/HeadToHead'
        dataset_print = 'HeadToHead'
    elif dataset[:16].lower() == 'faculty_business':
        hidden = 8
        hidden_compare = 8
        F = 6
        K = 5
        dataset = 'FacultyHiringNetworks/Business/Business_FM_Full_'
        dataset_print = 'Faculty: Business'
    elif dataset[:10].lower() == 'faculty_cs':
        hidden = 8
        hidden_compare = 8
        F = 8
        K = 9
        dataset = 'FacultyHiringNetworks/ComputerScience/ComputerScience_FM_Full_'
        dataset_print = 'Faculty: CS'
    elif dataset[:15].lower() == 'faculty_history':
        hidden = 8
        hidden_compare = 8
        F = 22
        K = 12
        dataset = 'FacultyHiringNetworks/History/History_FM_Full_'
        dataset_print = 'Faculty: History'
    elif dataset[:8].lower() == 'football':
        hidden = 4
        hidden_compare = 4
        F = 19
        K = 9
        dataset = 'Football_data_England_Premier_League/England_' + str(season) + '_' + str(season+1)
        dataset_print = 'Football({})'.format(season)
    elif dataset[:14].lower() == 'finer_football':
        hidden = 4
        hidden_compare = 4
        F = 4
        K = 9
        dataset = 'Football_data_England_Premier_League/finerEngland_' + str(season) + '_' + str(season+1)
        dataset_print = 'Football finer({})'.format(season)
    return '{\it '+dataset_print+'}', dataset, K, hidden

In [ ]:
lr_list = [0.01]
upset_margin_coeff_list = [0, 1]
trainable_alpha_list = [True]
rank_baseline_list = ['serialRank', 'SpringRank','syncRank','btl', 
        'eigenvectorCentrality', 'PageRank', 'SVD_NRS']
upset_ratio_coeff_list = [0, 1]
Fiedler_layer_num_list = [5]
pretrain_epochs_list = [50]
non_proximal_ind_correspondence_dict = {}
proximal_ind_correspondence_dict = {}
i = 0
for lr_ind, lr in enumerate(lr_list):
    for train_ind, train_with in enumerate(train_with_list):
        for pretrain_ind, pretrain_with in enumerate(pretrain_with_list_dict[train_with]):
            for margin_coeff_ind, upset_margin_coeff in enumerate(upset_margin_coeff_list):
                for trainable_alpha_ind, trainable_alpha in enumerate(trainable_alpha_list):
                    for base_ind, rank_baseline in enumerate(rank_baseline_list):
                        for ratio_coeff_ind, upset_ratio_coeff in enumerate(upset_ratio_coeff_list):
                            for Fiedler_layer_num_ind, Fiedler_layer_num in enumerate(Fiedler_layer_num_list):
                                for pretrain_epochs_ind, pretrain_epochs in enumerate(pretrain_epochs_list):
                                    for method_ind in range(NON_PROXIMAL_GNN_NUM):
                                        non_proximal_ind_correspondence_dict[i] = [lr_ind, train_ind, pretrain_ind, \
                                                                        margin_coeff_ind, trainable_alpha_ind, \
                                                                        base_ind, ratio_coeff_ind, \
                                                                    Fiedler_layer_num_ind, pretrain_epochs_ind, method_ind]
                                        i += 1
i = 0
for lr_ind, lr in enumerate(lr_list):
    for train_ind, train_with in enumerate(train_with_list):
        for pretrain_ind, pretrain_with in enumerate(pretrain_with_list_dict[train_with]):
            for margin_coeff_ind, upset_margin_coeff in enumerate(upset_margin_coeff_list):
                for trainable_alpha_ind, trainable_alpha in enumerate(trainable_alpha_list):
                    for base_ind, rank_baseline in enumerate(rank_baseline_list):
                        for ratio_coeff_ind, upset_ratio_coeff in enumerate(upset_ratio_coeff_list):
                            for Fiedler_layer_num_ind, Fiedler_layer_num in enumerate(Fiedler_layer_num_list):
                                for pretrain_epochs_ind, pretrain_epochs in enumerate(pretrain_epochs_list):
                                    for method_ind in range(PROXIMAL_GNN_NUM):
                                        proximal_ind_correspondence_dict[i] = [lr_ind, train_ind, pretrain_ind, \
                                                                        margin_coeff_ind, trainable_alpha_ind, \
                                                                        base_ind, ratio_coeff_ind, \
                                                                    Fiedler_layer_num_ind, pretrain_epochs_ind, method_ind]
                                        i += 1
non_proximal_cases_num = len(non_proximal_ind_correspondence_dict.keys())
proximal_cases_num = len(proximal_ind_correspondence_dict.keys())
print(non_proximal_cases_num, proximal_cases_num)

In [ ]:
def extract_results(dataset, season=2009, K=5, upset_ratio_coeff=1.0, upset_margin=0.01, p=0.1, 
                       AllTrain=True, eta=0.1, lr=0.05, hidden=32, 
                        N=350, ERO_style='uniform', train_ratio = 0.8, test_ratio = 0.1, dropout=0.5, sigma=1.0, 
                           methods_of_interest=['Non_proximal', 'Proximal'], print_latex=True):
    F_style = 'path'
    sp_style = 'random'
    num_trials = 2
    seed = 31
    fill_val = 0.5
    ambient = 0
    alpha = 1
    seeds = [10, 20, 30, 40, 50]
    
    dataset_print, dataset, K, hidden = dataset_and_K(dataset, season, K, p, ERO_style, eta)

    if dataset[:3] != 'ERO':
        num_trials = 10
        AllTrain = True
        train_ratio = 1
        test_ratio = 1
        seeds = [10]
    

    kendalltau_res_full, final_upset_full, selected_indices = GNN_selection(dataset, K, train_ratio, test_ratio, AllTrain, hidden, num_trials)
    dir_name = '../result_arrays/'+dataset
    kendalltau_res_all = np.zeros((METRICS_NUM, METHODS_NUM, num_trials*len(seeds)))
    kendalltau_res_all[:] = np.nan
    final_upset_all = np.zeros((METRICS_NUM, METHODS_NUM, num_trials*len(seeds), NUM_UPSET_CHOICES))
    final_upset_all[:] = np.nan
    for i in range(METRICS_NUM):
        compare_names_all = ['Non_proximal', 'Proximal']
        if i == 0 and dataset[:3] != 'ERO':
            continue
        kendalltau_res = kendalltau_res_full[i]
        final_upset = final_upset_full[i]

        # try to include mvr results
        save_name = generate_save_name(dataset=dataset, all_methods=mvr, K=K, num_trials=num_trials, 
                                       train_ratio=train_ratio, test_ratio=test_ratio,  AllTrain=AllTrain)
        method_str, _ = generate_method_str_and_compare_names_all(mvr)
        if os.path.exists(os.path.join(dir_name,'kendalltau',method_str,save_name) + '.npy'):
            kendalltau_res = np.concatenate((np.load(os.path.join(dir_name,'kendalltau',method_str,save_name) + '.npy')[:, :, 2, 0], kendalltau_res), axis=0)
            final_upset = np.concatenate((np.load(os.path.join(dir_name,'upset',method_str,save_name) + '.npy'), final_upset), axis=0)
        else:
            fill_nan = np.zeros((1, num_trials*len(seeds)))
            fill_nan[:] = np.nan
            kendalltau_res = np.concatenate((fill_nan, kendalltau_res), axis=0)
            fill_nan = np.zeros((1, num_trials*len(seeds), NUM_UPSET_CHOICES))
            fill_nan[:] = np.nan
            final_upset = np.concatenate((fill_nan, final_upset), axis=0)
        compare_names_all = ['mvr'] + compare_names_all


        # include baseline results
        save_name = generate_save_name(dataset=dataset, all_methods=baselines, K=K, num_trials=num_trials, 
                                       train_ratio=train_ratio, test_ratio=test_ratio,  AllTrain=AllTrain)
        method_str, compare_names_baselines = generate_method_str_and_compare_names_all(baselines)
        kendalltau_res = np.concatenate((np.load(os.path.join(dir_name,'kendalltau',method_str,save_name) + '.npy')[:, :, 2, 0], kendalltau_res), axis=0)
        final_upset = np.concatenate((np.load(os.path.join(dir_name,'upset',method_str,save_name) + '.npy'), final_upset), axis=0)
        compare_names_all = compare_names_baselines + compare_names_all

        kendalltau_res_all[i] = kendalltau_res
        final_upset_all[i] = final_upset
        
    return dataset_print, kendalltau_res_all, final_upset_all, selected_indices

In [ ]:
def GNN_load_results(dataset=dataset, all_methods=all_GNNs, K=5, train_with='dist', 
                                   upset_ratio_coeff=1, upset_margin_coeff=0,  upset_margin=0.01,
                                    trainable_alpha=False, lr=0.01, hidden=32, num_trials=10, 
                                   train_ratio=0.8, test_ratio=0.1,  AllTrain=True, rank_baseline='SpringRank', 
                                   sigma=1, Fiedler_layer_num=5, pretrain_epochs=50, pretrain_with='dist'):
    save_name = generate_save_name(dataset=dataset, all_methods=all_GNNs, K=K, train_with=train_with, 
                                   upset_ratio_coeff=upset_ratio_coeff, upset_margin_coeff=upset_margin_coeff, upset_margin=upset_margin,
                                    trainable_alpha=trainable_alpha, lr=lr, hidden=hidden, num_trials=num_trials, 
                                   train_ratio=train_ratio, test_ratio=test_ratio,  AllTrain=AllTrain, rank_baseline=rank_baseline, 
                                   sigma=sigma, Fiedler_layer_num=Fiedler_layer_num, pretrain_epochs=pretrain_epochs, pretrain_with=pretrain_with)
    method_str, _ = generate_method_str_and_compare_names_all(all_GNNs)
    dir_name = '../result_arrays/'+dataset
    kendalltau_res = None
    if dataset[:3] == 'ERO':
        kendalltau_res = np.load(os.path.join(dir_name,'kendalltau',method_str,save_name) + '.npy')[:, :, 2, 0]
    final_upset = np.load(os.path.join(dir_name,'upset',method_str,save_name) + '.npy')
    return kendalltau_res, final_upset

In [ ]:
def GNN_selection(dataset, K, train_ratio, test_ratio, AllTrain, hidden, num_trials):
    upset_margin = 0.01
    pretrain_epochs = 50
    sigma = 1
    full_results_non_proximal = 1000*np.ones((METRICS_NUM, NON_PROXIMAL_GNN_NUM * non_proximal_cases_num))
    full_results_non_proximal[0] = 0
    full_results_proximal = 1000*np.ones((METRICS_NUM, PROXIMAL_GNN_NUM * proximal_cases_num))
    full_results_proximal[0] = 0
    final_ind_non_proximal = 0
    final_ind_proximal = 0
    has_result = False
    for lr_ind, lr in enumerate(lr_list):
        for train_ind, train_with in enumerate(train_with_list):
            for pretrain_ind, pretrain_with in enumerate(pretrain_with_list_dict[train_with]):
                for margin_coeff_ind, upset_margin_coeff in enumerate(upset_margin_coeff_list):
                    for trainable_alpha_ind, trainable_alpha in enumerate(trainable_alpha_list):
                        for base_ind, rank_baseline in enumerate(rank_baseline_list):
                            for ratio_coeff_ind, upset_ratio_coeff in enumerate(upset_ratio_coeff_list):
                                for Fiedler_layer_num_ind, Fiedler_layer_num in enumerate(Fiedler_layer_num_list):
                                    for pretrain_epochs_ind, pretrain_epochs in enumerate(pretrain_epochs_list):
                                        try:
                                            kendalltau, upsets = GNN_load_results(dataset=dataset, all_methods=all_GNNs, K=K, train_with=train_with, 
                                                    upset_ratio_coeff=upset_ratio_coeff, upset_margin_coeff=upset_margin_coeff, upset_margin=upset_margin,
                                                    trainable_alpha=trainable_alpha, lr=lr, hidden=hidden, num_trials=num_trials, 
                                                    train_ratio=train_ratio, test_ratio=test_ratio,  AllTrain=AllTrain, rank_baseline=rank_baseline, 
                                                    sigma=sigma, Fiedler_layer_num=Fiedler_layer_num, pretrain_epochs=pretrain_epochs, pretrain_with=pretrain_with)
                                            if kendalltau is not None:
                                                mean_kendalltau = np.nanmean(kendalltau[non_proximal_bool], axis=1)
                                                full_results_non_proximal[0, final_ind_non_proximal: final_ind_non_proximal + NON_PROXIMAL_GNN_NUM] = mean_kendalltau
                                                mean_kendalltau = np.nanmean(kendalltau[proximal_bool], axis=1)
                                                full_results_proximal[0, final_ind_proximal: final_ind_proximal + PROXIMAL_GNN_NUM] = mean_kendalltau

                                            mean_upsets = upsets[non_proximal_bool].swapaxes(0,-1).mean(axis=1)
                                            full_results_non_proximal[1:, final_ind_non_proximal: final_ind_non_proximal + NON_PROXIMAL_GNN_NUM] = mean_upsets
                                            mean_upsets = upsets[proximal_bool].swapaxes(0,-1).mean(axis=1)
                                            full_results_proximal[1:, final_ind_proximal: final_ind_proximal + PROXIMAL_GNN_NUM] = mean_upsets
                                            has_result = True
                                        except FileNotFoundError:
                                            pass
                                        except AssertionError:
                                            pass
                                        except ValueError:
                                            pass
                                        final_ind_non_proximal += NON_PROXIMAL_GNN_NUM
                                        final_ind_proximal += PROXIMAL_GNN_NUM
    if has_result:
        best_ind = np.zeros((METRICS_NUM, 2))
        best_vals = np.zeros((METRICS_NUM, 2))
        best_vals[:] = np.nan
        if dataset[:3] == 'ERO':
            full_results_non_proximal[0] = np.nan_to_num(full_results_non_proximal[0], nan=0)
            best_ind[0, 0] = full_results_non_proximal[0].argmax()
            best_vals[0, 0] = np.nanmax(full_results_non_proximal[0])
            full_results_proximal[0] = np.nan_to_num(full_results_proximal[0], nan=0)
            best_ind[0, 1] = full_results_proximal[0].argmax()
            best_vals[0, 1] = np.nanmax(full_results_proximal[0])
        full_results_non_proximal[1:] = np.nan_to_num(full_results_non_proximal[1:], nan=1000)
        full_results_proximal[1:] = np.nan_to_num(full_results_proximal[1:], nan=1000)
        
        for upset_ind in range(NUM_UPSET_CHOICES):
            best_vals[1+upset_ind, 0] = np.nanmin(full_results_non_proximal[1+upset_ind])
            best_ind[1+upset_ind, 0] = full_results_non_proximal[1+upset_ind].argmin()
            best_vals[1+upset_ind, 1] = np.nanmin(full_results_proximal[1+upset_ind])
            best_ind[1+upset_ind, 1] = full_results_proximal[1+upset_ind].argmin()
        selected_indices = np.zeros((METRICS_NUM, 2, GNN_CHOICES_NUM+1))
        kendalltau_res = np.zeros((METRICS_NUM, 2, 10))
        kendalltau_res[:] = np.nan
        final_upset = np.zeros((METRICS_NUM, 2, 10, NUM_UPSET_CHOICES)) # the first "2" means non-proximal and proximal
        final_upset[:] = np.nan
        for i in range(METRICS_NUM):
            if i == 0 and dataset[:3] != 'ERO':
                continue
            for j, proximal_or_not in enumerate(['non_proximal', 'proximal']):
                if j == 0:
                    selected_indices[i, j] = non_proximal_ind_correspondence_dict[best_ind[i, j]]
                else:
                    selected_indices[i, j] = proximal_ind_correspondence_dict[best_ind[i, j]]
                
                lr = lr_list[int(selected_indices[i, j, 0])]
                train_with = train_with_list[int(selected_indices[i, j, 1])]
                pretrain_with = pretrain_with_list_dict[train_with][int(selected_indices[i, j, 2])]
                upset_margin_coeff = upset_margin_coeff_list[int(selected_indices[i, j, 3])]
                trainable_alpha = trainable_alpha_list[int(selected_indices[i, j, 4])]
                rank_baseline = rank_baseline_list[int(selected_indices[i, j, 5])]
                upset_ratio_coeff = upset_ratio_coeff_list[int(selected_indices[i, j, 6])]
                Fiedler_layer_num = Fiedler_layer_num_list[int(selected_indices[i, j, 7])]
                pretrain_epochs = pretrain_epochs_list[int(selected_indices[i, j, 8])]

                sel_ind = int(selected_indices[i, j, -1])
                if j == 0:
                    GNN_selected = GNN_names_non_proximal[sel_ind]
                else:
                    GNN_selected = GNN_names_proximal[sel_ind]
                selected_vals = [lr, train_with, pretrain_with, upset_margin_coeff, \
                                  trainable_alpha, \
                                 rank_baseline, upset_ratio_coeff, Fiedler_layer_num, pretrain_epochs]
                print_str = GNN_selected + ' on ' + dataset + ' among ' + proximal_or_not + ': '
                for k in range(GNN_CHOICES_NUM):
                    print_str += GNN_selection_choices[k] + '=' + str(selected_vals[k]) + ', '
                print_str += 'for the best ' + selected_metrics[i]
                kendalltau, upsets = GNN_load_results(dataset=dataset, all_methods=all_GNNs, K=K, train_with=train_with, 
                                           upset_ratio_coeff=upset_ratio_coeff, upset_margin_coeff=upset_margin_coeff, upset_margin=upset_margin,
                                            trainable_alpha=trainable_alpha, lr=lr, hidden=hidden, num_trials=num_trials, 
                                           train_ratio=train_ratio, test_ratio=test_ratio,  AllTrain=AllTrain, rank_baseline=rank_baseline, 
                                           sigma=sigma, Fiedler_layer_num=Fiedler_layer_num, pretrain_epochs=pretrain_epochs, pretrain_with=pretrain_with)
                if j == 0:
                    if kendalltau is not None:
                        kendalltau_res[i, j] = (kendalltau[non_proximal_bool])[sel_ind]
                    upsets_res = (upsets[non_proximal_bool])[sel_ind]
                else:
                    if kendalltau is not None:
                        kendalltau_res[i, j] = (kendalltau[proximal_bool])[sel_ind]
                    upsets_res = (upsets[proximal_bool])[sel_ind]
                final_upset[i, j] = np.array(upsets_res)
        return kendalltau_res, final_upset, selected_indices
    else:
        raise FileNotFoundError

In [ ]:
def analysis_results(dataset_list=['HeadToHead', 'finance','animal', 'faculty_business', \
                                   'faculty_cs', 'faculty_history', \
                                   'avg_basketball', 'avg_finer_basketball', 'avg_football', 'avg_finer_football', \
                                  'basketball', 'finer_basketball', 'football', 'finer_football'], selected_printing=False):
    dataset_name_full = []
    kendalltau_res_all_full = []
    final_upset_all_full = []
    for dataset in dataset_list:
        if dataset[:4] == 'avg_':
            kendalltau_res_all_full_avg = []
            final_upset_all_full_avg = []
            for season in season_list_dict[dataset[4:]]:
                _, kendalltau_res_all, final_upset_all, _ = extract_results(dataset=dataset[4:], 
                                                        season=season)
                kendalltau_res_all_full_avg.append(kendalltau_res_all)
                final_upset_all_full_avg.append(final_upset_all)
            dataset_name_full.append(name_mapping_dict[dataset])
            kendalltau_res_all_full.append(np.nanmean(np.array(kendalltau_res_all_full_avg), axis=0))
            final_upset_all_full.append(np.nanmean(np.array(final_upset_all_full_avg), axis=0))
        else:
            for p in p_list_dict[dataset]:
                for K in K_list_dict[dataset]:
                    for eta in eta_list_dict[dataset]:
                        for ERO_style in ERO_style_list_dict[dataset]:
                            for season in season_list_dict[dataset]:
                                if selected_printing:
                                    if ERO_style == 'uniform':
                                        if p == 0.05 and eta not in [0.1, 0.3]:
                                            continue
                                        elif p == 1 and eta not in [0.5, 0.7]:
                                            continue
                                    else:
                                        if p == 0.05 and eta not in [0.2, 0.4]:
                                            continue
                                        elif p == 1 and eta not in [0.6]:
                                            continue
                                try:
                                    dataset_long, kendalltau_res_all, final_upset_all, _ = extract_results(dataset=dataset, 
                                                    season=season, p=p, K=K, eta=eta,
                                                    ERO_style=ERO_style)
                                    dataset_name_full.append(dataset_long)
                                    kendalltau_res_all_full.append(kendalltau_res_all)
                                    final_upset_all_full.append(final_upset_all)
                                except FileNotFoundError:
                                    print('No result yet for {}, season {}, p={}, K={}, eta={}, ERO style = {}.'.format(dataset,
                                        season, p, K, eta, ERO_style))
    

    full_results = np.concatenate((np.expand_dims(np.array(kendalltau_res_all_full), axis=-1), np.array(final_upset_all_full)), axis=-1)
    for i in range(METRICS_NUM):
        for j in range(1, METRICS_NUM):
            results_to_print = full_results[:,j,:,:,i].swapaxes(0,2)
            if not np.isnan(results_to_print).all() and (i==j or i==0):
                dataset_name_print = dataset_name_full
                compare_names_print = compare_names_all
                if i not in [1, 3]:
                    mvr_ind = compare_names_all.index('mvr')
                    results_to_print = np.delete(results_to_print, mvr_ind, axis=1)
                    compare_names_print = np.delete(compare_names_print, mvr_ind, axis=0)
                if selected_printing:
                    results_to_print_mean = np.nanmean(results_to_print, axis=0)
                    if i == 0: # the bigger, the better
                        thres =  -np.sort(-results_to_print_mean, axis=0)[1]
                        bool_vals = np.bitwise_and(results_to_print_mean[-1]>=thres, results_to_print_mean[-1] >= results_to_print_mean[-2])
                        bool_vals = np.bitwise_and(results_to_print_mean[-1] >= results_to_print_mean[-2], results_to_print_mean[-1] >= results_to_print_mean[0])
                        selected_ind = np.arange(results_to_print.shape[-1])[bool_vals]
                    else:
                        thres = np.sort(results_to_print_mean, axis=0)[1]
                        bool_vals = np.bitwise_and(results_to_print_mean[-1]<=thres, results_to_print_mean[-1] <= results_to_print_mean[-2])
                        bool_vals = np.bitwise_and(results_to_print_mean[-1] <= results_to_print_mean[-2], results_to_print_mean[-1] <= results_to_print_mean[0])
                        selected_ind = np.arange(results_to_print.shape[-1])[bool_vals]
                    results_to_print = np.take(results_to_print, selected_ind, axis=-1)
                    dataset_name_print = np.take(dataset_name_print, selected_ind)
                title_name = selected_metrics[i] + ' with best ' + selected_metrics[j]
                print_overall_performance_mean_std(title_name, results_to_print, 
                                compare_names_print, dataset_name_print, True)

In [ ]:
analysis_results(['ERO'], True)

In [ ]:
analysis_results(['ERO'], False)

In [ ]:
analysis_results()

In [ ]:
def analysis_models(dataset_list=['HeadToHead', 'finance','animal', 'faculty_business', 'faculty_cs', 'faculty_history', 'basketball', 'finer_basketball', 'football', 'finer_football']):
    dataset_name_full = []
    kendalltau_res_all_full = []
    final_upset_all_full = []
    selected_indices_full = []
    for dataset in dataset_list:
        for p in p_list_dict[dataset]:
            for K in K_list_dict[dataset]:
                for eta in eta_list_dict[dataset]:
                    for ERO_style in ERO_style_list_dict[dataset]:
                        for season in season_list_dict[dataset]:
                            try:
                                dataset_long, kendalltau_res_all, final_upset_all, selected_indices = extract_results(dataset=dataset, 
                                                season=season, p=p, K=K, eta=eta,
                                                ERO_style=ERO_style)
                                dataset_name_full.append(dataset_long)
                                kendalltau_res_all_full.append(kendalltau_res_all)
                                final_upset_all_full.append(final_upset_all)
                                selected_indices_full.append(selected_indices)
                            except FileNotFoundError:
                                print('No result yet for {}, season {}, p={}, K={}, eta={}, ERO style = {}.'.format(dataset,
                                    season, p, K, eta, ERO_style))
    
    full_results = np.concatenate((np.expand_dims(np.array(kendalltau_res_all_full), axis=-1), np.array(final_upset_all_full)), axis=-1)
    for i in range(METRICS_NUM):
        for j, proximal_or_not in enumerate(['non_proximal', 'proximal']):
            title = 'GNN selection among {} for the best {}.'.format(proximal_or_not, selected_metrics[i])
            label = 'GNN_selection_'+proximal_or_not+'_'+selected_metrics[i]
            print(title)
            t = Texttable(max_width=120)
            t.set_deco(Texttable.HEADER)
            t.add_row(['Data set', 'Variant'] + GNN_selection_choices_curr)
            for dataset_ind, dataset in enumerate(dataset_name_full):
                if i == 0 and dataset[:3] != 'ERO':
                    continue
                selected_indices = selected_indices_full[dataset_ind]

                lr = lr_list[int(selected_indices[i, j, 0])]
                train_with = train_with_list[int(selected_indices[i, j, 1])]
                pretrain_with = name_mapping_dict[pretrain_with_list_dict[train_with][int(selected_indices[i, j, 2])]]
                train_with = name_mapping_dict[train_with]
                upset_margin_coeff = upset_margin_coeff_list[int(selected_indices[i, j, 3])]
                trainable_alpha = str(trainable_alpha_list[int(selected_indices[i, j, 4])])
                rank_baseline = name_mapping_dict[rank_baseline_list[int(selected_indices[i, j, 5])]]
                upset_ratio_coeff = upset_ratio_coeff_list[int(selected_indices[i, j, 6])]
                Fiedler_layer_num = Fiedler_layer_num_list[int(selected_indices[i, j, 7])]
                pretrain_epochs = pretrain_epochs_list[int(selected_indices[i, j, 8])]
                if train_with[:8] != 'proximal':
                    pretrain_with = '--'
                    trainable_alpha = '--'
                    pretrain_epochs = '--'
                
                

                sel_ind = int(selected_indices[i, j, -1])
                if j == 0:
                    GNN_selected = GNN_names_non_proximal[sel_ind]
                else:
                    GNN_selected = GNN_names_proximal[sel_ind]
                if train_with[:8] != 'proximal' and GNN_selected not in GNN_names_proximal:
                    Fiedler_layer_num = '--'
                GNN_selected = name_mapping_dict[GNN_selected]
                
                if train_with != 'proximal baseline' and GNN_selected not in ['DIGRAC proximal baseline', 'ib proximal baseline']:
                    rank_baseline = '--'


                selected_vals = [train_with, pretrain_with, upset_margin_coeff, \
                                 rank_baseline, upset_ratio_coeff]
                t.add_row([dataset, GNN_selected] + selected_vals)
            print(t.draw())
            print(latextable.draw_latex(t, caption=title, label="table:"+label) + "\n")

In [ ]:
analysis_models()

In [ ]:
analysis_models(['ERO'])